In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [1]:
import sys
#sys.path.append('/content/drive/My Drive/APT/newest/TAR-project-master')
sys.path.append('/Users/patrik/PycharmProjects/TAR-project')

In [2]:
pip install num2words

Error processing line 1 of /opt/anaconda3/lib/python3.7/site-packages/matplotlib-3.1.3-py3.7-nspkg.pth:

  Traceback (most recent call last):
    File "/opt/anaconda3/lib/python3.7/site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 580, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install langdetect

Error processing line 1 of /opt/anaconda3/lib/python3.7/site-packages/matplotlib-3.1.3-py3.7-nspkg.pth:

  Traceback (most recent call last):
    File "/opt/anaconda3/lib/python3.7/site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 580, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored
  Using cached langdetect-1.0.8.tar.gz (981 kB)
  Created wheel for langdetect: filename=langdetect-1.0.8-py3-none-any.whl size=993191 sha256=5a2750483c8bf03815d5d7bebd172cbf73f9a79d297c9ba6e8aae7721d9048dc
  Stored in directory: /Users/patrik/Library/Caches/pip/wheels/59/f6/9d/85068904dba861c0b9af74e286265a08da438748ee5ae56067
Successfully built langdetect
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install -U sentence-transformers

Error processing line 1 of /opt/anaconda3/lib/python3.7/site-packages/matplotlib-3.1.3-py3.7-nspkg.pth:

  Traceback (most recent call last):
    File "/opt/anaconda3/lib/python3.7/site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 580, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored
  Using cached sentence-transformers-0.2.6.1.tar.gz (55 kB)
     |████████████████████████████████| 660 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 4.2 MB/s eta 0:00:01
  Using cached sacremoses-0.0.43.tar.gz (883 kB)
  Using cached tokenizers-0.7.0-cp37-cp37m-macosx_10_10_x86_64.whl (1.2 MB)
     |████████████████████████████████| 696 kB 3.5 MB/s eta 0:00:01
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.6.1-py3-none-any.whl size=74030 sha256=3e28be2abc0b23f902fdf09f0a47518e434ec778da9aeed9f7af0c2e2bac

In [2]:
import pandas as pd
import numpy as np
import nltk

nltk.download('stopwords')
nltk.download('punkt')

from dataset.preprocessing.preprocessing import word_stem

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/patrik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/patrik/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
#abstracts_path = '/content/drive/My Drive/APT/temp_data/cvite_data/body_texts.csv'
abstracts_path = '/Users/patrik/PycharmProjects/TAR-project/data/corpus.csv'

abstracts = pd.read_csv(abstracts_path)
abstracts.head()

,Unnamed: 0,paper_id,section,text,preprocessed_text
0,0,a5729379cb2d18730da32cf117283be2cc16cf05,"Dear Editor,","In December 2019, a novel human coronavirus ca...",decemb two thousand and nineteen novel human c...
1,1,a5729379cb2d18730da32cf117283be2cc16cf05,"Dear Editor,",The nomenclature for this coronavirus is still...,the nomenclatur coronaviru still controversi ....
2,2,a5729379cb2d18730da32cf117283be2cc16cf05,"Dear Editor,",Coronaviruses are enveloped non-segmented posi...,coronavirus envelop non-seg posit sens rna vir...
3,3,a5729379cb2d18730da32cf117283be2cc16cf05,"Dear Editor,",Receptor recognition is an important factor de...,receptor recognit import factor determin host ...
4,4,a5729379cb2d18730da32cf117283be2cc16cf05,"Dear Editor,","In this study, we discovered one deletion and ...",studi discov one delet three insert 2019-ncov ...


In [0]:
dupl = abstracts[abstracts.duplicated(subset='preprocessed_text')]

In [0]:
dupl.head().style.set_properties(**{'font-size': '9pt'})

## Queries

In [6]:
queries2 = [
           'What is known about transmission, incubation, and environmental stability?',
           'What do we know about natural history, transmission, and diagnostics for the virus?',
           'What have we learned about infection prevention and control?',
           "incubation period of COVID-19",
           "Does smoking increase risks when having covid19?",
           "mortality rate of covid19?",
           "pregnancy complications covid19?"
]

queries = [
           'Does smoking affect severness of covid19 infection?',
          'How are pregnant women affected by covid19?',
          'Co-morbidities affecting covid19',
          'What is the fatality of covid19?',
          'What is the basic reproductive number of covid19?',
          'What is the incubation period of covid19?',
          'Most common ways of transmission of covid19',
          'Are enviromental factors affecting transmission of covid19?'
]


queries3 = [
           'Does smoking affect severness of covid19 infection?',
          'pregnant women affected by covid19?',
          'Co-morbidities affecting covid19',
          'fatality of covid19?',
          'basic reproductive number of covid19?',
          'incubation period of covid19?',
          'transmission of covid19',
          'Are enviromental factors affecting transmission of covid19?'
]

queries = [
    'Smoking effect on the severity of covid19',
    'Incubation period of covid19',
    'Covid19 effect on pregnancy',
    'Mortality rate of covid19',
    'Comorbidities and coinfections with effect on the severity of covid19',
    'Temperature effect on covid19',
    'Ways of covid19 transmission',
    'Basic reproductive number of covid19'
]


color_list = ['black', 'red', 'blue', 'green']
colors = {}
for i in range(len(queries)):
    colors[queries[i]] = color_list[i % len(color_list)]


def run_queries(query_engine, queries, k=5):
    results = query_engine.run_query(queries[0], k)
    if len(queries) > 1:
        for q in queries[1:]:
            results = pd.concat([results, query_engine.run_query(q, k)])
    return results

def color_rows(val):
    col = colors[val['query']]
    return ['color: %s' % col]*len(val)

In [7]:
all_results = []
num_of_results = 10

## TF-IDF

In [8]:
from nltk.corpus import stopwords
from query_model.query_engines import BOWQueryEngine
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [9]:
from dataset.preprocessing.tokenizers import StemmTokenizer

In [12]:
stop_words = set(stopwords.words('english'))
cv = CountVectorizer(stop_words=stop_words)
transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
query_engine = BOWQueryEngine(cv, transformer)
query_engine.fit(abstracts)

In [0]:
# If you want to load already trained model
# query_engine = BOWQueryEngine.load('/content/drive/My Drive/APT/trained_models/tf_idf.dat')

In [19]:
#query_engine.save('/Users/patrik/PycharmProjects/TAR-project/trained_models/', 'tf_idf')

Writing object to /Users/patrik/PycharmProjects/TAR-project/trained_models/tf_idf.dat


In [10]:
res = run_queries(query_engine, queries, k=num_of_results)
res.reset_index(drop=True)[['query', 'text', 'sim']].style.set_properties(**{'font-size': '9pt'}).apply(color_rows, axis=1)

100%|██████████| 1/1 [00:00<00:00, 673.03it/s]


st ['smoke effect sever covid19 ']


100%|██████████| 1/1 [00:00<00:00, 552.46it/s]


st ['incub period covid19 ']


100%|██████████| 1/1 [00:00<00:00, 763.99it/s]


...

100%|██████████| 1/1 [00:00<00:00, 766.78it/s]


st ['way covid19 transmiss ']


100%|██████████| 1/1 [00:00<00:00, 596.97it/s]


st ['basic reproduct number covid19 ']


,query,text,sim
0,Smoking effect on the severity of covid19,"Hence, viral DNA content seems to be higher in laser and ultrasonic smoke than in electrocoagulation smoke (25) . Since surgical smoke contains many viruses(4,14,15) many of the surgeons acquired their infection through it (21) . In this context, we cannot ignore the possibility of spreading COVID-19 through cellular particles in surgical smoke.",0.371692
1,Smoking effect on the severity of covid19,"We need to receive education about the hazards of smoke plume to raise awareness of the health risks of surgical smoke. Each of the many precautions currently available has its own limitations, so smoke extraction devices are very much recommended than any other. It is highly recommended to follow the standardised guidelines for surgical smoke safety. Since it has been proven that surgical smoke carries full virus particle(such as COVID-19 virus), it is strongly recommended to minimise or avoid electrocautery during the COVID-19 outbreak. Surgical smoke poses numerous risks to the surgeon, including the transmission of infectious diseases, mutagenicity, and direct physical injury.",0.367093
2,Smoking effect on the severity of covid19,"Background: Many surgeons are unaware of the risks posed by the surgical diathermy. Apart from the numerous chemicals, surgical smoke had been shown to harbour intact bacterial and virus particles especially COVID-19 in the current time. Objective: To identify the inhalational, infectious, chemical, and mutagenic risks of surgical smoke and suggest evidence-based hazard reduction strategies. Also to cogitate on the very high risk of viral spread spread by the use of surgical diathermy in COVID-19 outbreak. Methods: A review of articles indexed for MEDLINE on PubMed using the keywords surgical smoke, diathermy, electrocautery, surgical smoke hazards, smoke evacuator, and guidelines for surgical smoke safety was performed. The review included evidences from 50 articles from the dermatology, surgery, infectious disease, obstetrics, and cancer biology literature. There are risks associated with surgical smoke. Although some surgeons were aware, majority were not keen in the hazard reduction strategies. Many chemical and biological particles have been found in surgical smoke. It is highly recommended to follow the standardised guidelines for surgical smoke safety. Surgical smoke carries full virus particle(such as COVID-19 virus), it is strongly recommended to minimise or avoid electrocautery during the COVID-19 outbreak.",0.338894
3,Smoking effect on the severity of covid19,"Studies have shown similarities between surgical smoke and the adverse effects of air pollution and passive cigarette smoke (5, 13) . Surgical smoke also contains a variety of cellular particles which harbour contagious, viable malignant cells, and even to contain live bacteria and viruses. Different studies have clearly mentioned many viruses like HIV, infectious polio virus, human papilloma virus(HPV) and hepatitis B virus (HBV) are present in surgical smoke (3, 4, 14, 15) , this knowledge is relevant in COVID-19 outbreak.. There for, it is possible to have viruses like COVID-19 in cellular particles.",0.316200
4,Smoking effect on the severity of covid19,"At first glance, the predicted numbers of preventable infections seem surprisingly high compared to the confirmed number of 769 positive COVID19 cases in WA 3 and 4661 confirmed positive cases in the entire US. However, the growth rate of COVID19 in the US has been rapid, increasing from 69 cases on 2/29/20 to 4661 confirmed cases just 17 days later. 2 Due to the current state of COVID19 testing, US statistics on confirmed COVID19 cases may not be the most reliable, either. Per CDC guidelines that were last updated 3/9/2020, laboratory testing for COVID19 is only indicated for individuals who both develop respiratory symptoms consistent with COVID19 and meet additional criteria, such as being hospitalized, having certain 

In [12]:
res

,id,query,text,sim
115106,ce6dd396a841e68e830113e4c718918c00296ec8,Smoking effect on the severity of covid19Smoki...,"Hence, viral DNA content seems to be higher in...",0.382450
115109,ce6dd396a841e68e830113e4c718918c00296ec8,Smoking effect on the severity of covid19Smoki...,We need to receive education about the hazards...,0.381736
219675,ce6dd396a841e68e830113e4c718918c00296ec8,Smoking effect on the severity of covid19Smoki...,Background: Many surgeons are unaware of the r...,0.345049
115105,ce6dd396a841e68e830113e4c718918c00296ec8,Smoking effect on the severity of covid19Smoki...,Studies have shown similarities between surgic...,0.323773
53236,ef395bb81012eacc7628d0425be9a3a46a8ada9b,Smoking effect on the severity of covid19Smoki...,"At first glance, the predicted numbers of prev...",0.321240
...,...,...,...,...
89296,9f28ffc6d0ffd16df162075e55d9a76e09226957,Basic reproductive number of covid19Basic repr...,On 24 th March 2020 the UK introduced a range ...,0.375938
136489,9275fca658701f7e6af4f0514ca2a95f3fd2c5cd,Basic reproductive number of covid19Basic repr...,Stephanie colleagues did not find any evidence...,0.363070
102466,859eb40d59630f3d3ec9f0ded4f803098eee0fb3,Basic reproductive number of covid19Basic repr...,Thermal screening at airport has 54% COVID19 d...,0.360399
93882,2c42e5f5e8d926df3dd5008d202bd41b59138a4a,Basic reproductive number of covid19Basic repr...,"If k is greater than 1, then the disease will ...",0.358627


In [21]:
res['model'] = 'tf-idf'
all_results.append(res)

## BM25

In [11]:
from nltk.corpus import stopwords
from query_model.query_engines import BOWQueryEngine
from query_model.transformers.bm25 import BM25Transformer

In [12]:
stop_words = set(stopwords.words('english'))
cv = CountVectorizer(stop_words=None)
transformer = BM25Transformer()
query_engine = BOWQueryEngine(cv, transformer)
query_engine.fit(abstracts)

In [0]:
# If you want to load already trained model
# query_engine = BOWQueryEngine.load('/content/drive/My Drive/APT/trained_models/bm25.dat')

In [26]:
#query_engine.save('/content/drive/My Drive/APT/trained_models/', 'bm25')
#query_engine.save('/Users/patrik/PycharmProjects/TAR-project/trained_models/', 'bm25')

Writing object to /Users/patrik/PycharmProjects/TAR-project/trained_models/bm25.dat


In [13]:
res = run_queries(query_engine, queries, k=num_of_results)
res.reset_index(drop=True)[['query', 'text', 'sim']].style.set_properties(**{'font-size': '9pt'}).apply(color_rows, axis=1)

100%|██████████| 1/1 [00:00<00:00, 422.64it/s]


st ['smoke effect sever covid19 ']


100%|██████████| 1/1 [00:00<00:00, 580.45it/s]


st ['incub period covid19 ']


100%|██████████| 1/1 [00:00<00:00, 653.22it/s]


...

100%|██████████| 1/1 [00:00<00:00, 571.12it/s]


st ['way covid19 transmiss ']


100%|██████████| 1/1 [00:00<00:00, 671.09it/s]


st ['basic reproduct number covid19 ']


,query,text,sim
0,Smoking effect on the severity of covid19,"1. ""MERS"" OR ""middle east respiratory syndrome"" OR ""novel coronavirus"" OR ""novel coronavirus 2012"" 2. ""sever*""OR ""fatal*""OR ""death"" OR ""mortalit*"" 3. ""hospitalization"" OR ""intensive care"" OR ""ICU"" 4. 1 AND 2 AND 3",0.282545
1,Smoking effect on the severity of covid19,"Subsequently the virus was renamed the Sever Acute Respiratory Syndrome Coronavirus 2 (SARS-CoV-2) and the infectious disease it causes was tagged Coronavirus Disease 2019 . Hence, on 30th January 2020, the WHO declared the disease as a global pandemic.",0.271609
2,Smoking effect on the severity of covid19,"A total of 43 COVID-19 patients were enrolled in this study, including 29 mild patients (67.4%) and 14 sever patients (32.6%) according to above mentioned conditions. The 43 COVID-19 patients consisted of 21 female patients (48.8%) and 22 male patients (51.2%) .",0.255025
3,Smoking effect on the severity of covid19,"Over the course of December 2019, the health authority of Wuhan City, Hubei province, China reported a cluster of pneumonia cases of unknown etiology [1] . The Chinese researcher rapidly isolated Sever Acute Respiratory Syndrome Coronavirus 2 (SARS-CoV-2) from a patient on 7",0.252397
4,Smoking effect on the severity of covid19,Current COVID-19 patients are considered stable if the case is mild and no hospitalisation or oxygen therapy is required. Sever and critical cases are classified as unstable. Confirmed recovery is considered if the patient has been asymptomatic for at least 30 days after the last negative laboratory test.,0.249939
5,Smoking effect on the severity of covid19,"The study covered 154 cases of novel coronavirus patients from the 20) . The average age of the moderate group, the sever group, and the critical group is 63±13, 63±14, and 65±15 y, respectively. There is no statistical significance between gender and age among these 3 groups (P>0.05).",0.236239
6,Smoking effect on the severity of covid19,"The CDC has been guiding US healthcare providers on the care of COVID-19 patients. However, in the setting of a pandemic, it is of utmost importance to be able to sever the chain of transmission so that new cases can be reduced and allow the healthcare delivery system to catch up.",0.228579
7,Smoking effect on the severity of covid19,"The study covered 154 cases of novel coronavirus patients from the Renmin Hospital of Wuhan University between 22 January 2020 to 25 February 2020. The patients were divided into 3 groups according to the Diagnosis and Treatment of Novel Coronavirus Patients (the Fifth Pilot Ed.), moderate group (total 49, male 26 and female 23), sever group (total 61, male 34 and female 27), critical group (total 44, male 24 and female 20). The average age of the moderate group, the sever group, and the critical group is 63±13, 63±14, and 65±15 y, respectively. There is no statistical significance between gender and age among these 3 groups (P>0.05).",0.212250
8,Smoking effect on the severity of covid19,"Respiratory viral infections can trigger asthma attack which may lead to sever morbidity. In this report, using molecular methods, we show the chronological association between human coronavirus -HKU1 infection and asthma exacerbation in a two years and seven months old asthmatic girl who was not under treatment and was otherwise healthy.",0.207706
9,Smoking effect on the severity of covid19,"The incubation period for the SARS-CoV-2 ranges between 2 – 14 days [7]. Nonetheless, this incubation period varies according to disease severity [36]. No age is immune against COVID-19 infection; nevertheless, elderly people, pregnant females and those with impaired immunity are liable to more sever disease sequelae than the paediatric age group [37].",0.205903


In [29]:
res['model'] = 'bm25'
all_results.append(res)

## Word2Vec

In [30]:
from query_model.query_engines import W2VQueryEngine

In [0]:
params = {
    'min_count': 5,
    'size': 300, 
    'workers': 3, 
    'window': 3,
    'sg': 0,
    'hs': 0,
    'negative': 5
    }
query_engine = W2VQueryEngine(params)
query_engine.fit(abstracts)

In [33]:
# If you want to load already trained model
query_engine = W2VQueryEngine.load('/Users/patrik/PycharmProjects/TAR-project/trained_models/w2v.dat')

In [0]:
#query_engine.save('/content/drive/My Drive/APT/trained_models/', 'w2v_body_texts2')

Writing object to /content/drive/My Drive/APT/trained_models/w2v_body_texts.dat


In [34]:
res = run_queries(query_engine, queries, k=num_of_results)
res.reset_index(drop=True)[['query', 'text', 'sim']].style.set_properties(**{'font-size': '9pt'}).apply(color_rows, axis=1)

100%|██████████| 1/1 [00:00<00:00, 289.26it/s]


st ['smoke effect sever covid19 ']


AttributeError: 'W2VQueryEngine' object has no attribute '_W2VQueryEngine__create_query_result'

In [0]:
res['model'] = 'w2v'
all_results.append(res)

## w2v tf-idf

In [ ]:
query_engine = W2VQueryEngine.load('/content/drive/My Drive/APT/trained_models/w2v_body_texts.dat')

## Doc2Vec

In [0]:
from query_model.query_engines import D2VQueryEngine

In [0]:
params = { 
    'alpha': 0.025,
    'vector_size': 100,
    'min_count': 5, 
    'negative': 5, 
    'hs': 0, 
    'sample': 0,
    'epochs': 20,
    'window': 15
    }
query_engine = D2VQueryEngine(params)
query_engine.fit(abstracts)

 88%|████████▊ | 164222/187010 [3:39:07<34:30, 11.01it/s]

Buffered data was truncated after reaching the output size limit.

In [0]:
# If you want to load already trained model
query_engine = D2VQueryEngine.load('/content/drive/My Drive/APT/trained_models/d2v_body_texts2.dat')

In [0]:
#query_engine.save('/content/drive/My Drive/APT/trained_models/', 'd2v_body_texts3')
#query_engine.save('/content/drive/My Drive/APT/trained_models/', 'd2v15000')

Writing object to /content/drive/My Drive/APT/trained_models/d2v_body_texts3.dat


In [0]:
res = run_queries(query_engine, queries, k=num_of_results)
res.reset_index(drop=True)[['query', 'text', 'sim']].style.set_properties(**{'font-size': '9pt'}).apply(color_rows, axis=1)

In [0]:
res['model'] = 'd2v'
all_results.append(res)

## BERT

In [0]:
from sentence_transformers import SentenceTransformer
from query_model.query_engines import BERTQueryEngine

In [0]:
import torch
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [ ]:
query_engine2 = BERTQueryEngine()
query_engine2.fit(abstracts, text_column='preprocessed_text')

In [ ]:
# If you want to load already trained model
#query_engine = BERTQueryEngine.load('/content/drive/My Drive/APT/trained_models/BERT.dat')
query_engine = BERTQueryEngine.load('/content/drive/My Drive/APT/trained_models/BERT_on_texts.dat')

In [ ]:
query_engine2.save('/content/drive/My Drive/APT/trained_models/', 'BERT_on_preprocessed_texts_body_texts')

In [ ]:
res = run_queries(query_engine2, queries, k=num_of_results)
res.reset_index(drop=True)[['query', 'text', 'sim']].style.set_properties(**{'font-size': '9pt'}).apply(color_rows, axis=1)

In [ ]:
res['model'] = 'bert'
all_results.append(res)

## RESULTS

In [ ]:
all_results_df = pd.concat(all_results).sort_values(by=['query'])

In [ ]:
all_results_df.reset_index(drop=True)[['query', 'text', 'sim', 'model']].style.set_properties(**{'font-size': '9pt'}).apply(color_rows, axis=1)

In [ ]:
all_results_df['niksa'] = 0
all_results_df['cvite'] = 0
all_results_df['paco'] = 0

In [ ]:
all_results_df.to_csv('/content/drive/My Drive/APT/results.csv')

In [0]:
all_results_df.to_csv('/content/drive/My Drive/APT/results.csv')

100%|██████████| 187010/187010 [1:08:22<00:00, 45.58it/s]


In [0]:
# If you want to load already trained model
#query_engine = BERTQueryEngine.load('/content/drive/My Drive/APT/trained_models/BERT.dat')
query_engine = BERTQueryEngine.load('/content/drive/My Drive/APT/trained_models/BERT_on_texts.dat')

In [0]:
query_engine2.save('/content/drive/My Drive/APT/trained_models/', 'BERT_on_preprocessed_texts_body_texts')

Writing object to /content/drive/My Drive/APT/trained_models/BERT_on_preprocessed_texts_body_texts.dat


In [0]:
res = run_queries(query_engine2, queries)
res.reset_index(drop=True)[['query', 'text', 'sim']].style.set_properties(**{'font-size': '9pt'}).apply(color_rows, axis=1)

In [0]:
res['model'] = 'bert'
all_results.append(res)

## RESULTS

In [0]:
all_results_df = pd.concat(all_results).sort_values(by=['query'])

In [0]:
all_results_df.reset_index(drop=True)[['query', 'text', 'sim', 'model']].style.set_properties(**{'font-size': '9pt'}).apply(color_rows, axis=1)

In [0]:
all_results_df['niksa'] = 0
all_results_df['cvite'] = 0
all_results_df['paco'] = 0

In [0]:
all_results_df.to_csv('/content/drive/My Drive/APT/results.csv')

In [145]:
a = pd.read_csv('/Users/patrik/PycharmProjects/TAR-project/data/results.csv')
a = a[['id', 'query', 'text', 'sim', 'model']]
a

,id,query,text,sim,model
0,ccfe1455524bb1da264c8d868ecb5247ba869508,Basic reproductive number of covid19,. The next-generation matrix will be used to d...,0.815270,d2v
1,8c6e523654a7eab291bd150a6894986f76a30b4f,Basic reproductive number of covid19,⟨ Figure 1 is near here ⟩ According to the con...,0.920774,w2v_tf-idm
2,6b4ec0cb89bcb5a29dfb2328792ce64ba06ac553,Basic reproductive number of covid19,Studies such as [3] show that the effectivenes...,0.915905,w2v_tf-idm
3,530bd16984e4c400758aba6c13ab077b57446a1c,Basic reproductive number of covid19,Machine learning identifies basic and effectiv...,0.914700,w2v_tf-idm
4,096b4e8b2eb39917f0017276339be437650cc797,Basic reproductive number of covid19,• R 0 and R e : The basic reproduction number ...,0.907802,w2v_tf-idm
...,...,...,...,...,...
395,ce5f5bca2fc5a9261e33ba2d9da278ad48cb0401,Ways of covid19 transmission,The recent outbreak of Covid19 has required ur...,0.371468,tf-idf
396,babbb6bc1bb493349e28c6721f0c01ac9b3a93eb,Ways of covid19 transmission,Pre-existing lung disease and diabetes were co...,0.401094,tf-idf
397,670790eb6d158a57450705bcd87218bf479ed3be,Ways of covid19 transmission,The pandemic of COVID19 is ongoing. Till April...,0.401133,tf-idf
398,859eb40d59630f3d3ec9f0ded4f803098eee0fb3,Ways of covid19 transmission,Thermal screening at airport has 54% COVID19 d...,0.266427,bm25


In [146]:
b = pd.read_csv('/Users/patrik/PycharmProjects/TAR-project/data/final_results2.csv')
b = b[['query', 'text', 'niksa', 'cvite', 'paco']]
b

,query,text,niksa,cvite,paco
0,Basic reproductive number of covid19,. The next-generation matrix will be used to d...,1,1,1
1,Basic reproductive number of covid19,⟨ Figure 1 is near here ⟩ According to the con...,1,1,0
2,Basic reproductive number of covid19,Studies such as [3] show that the effectivenes...,1,1,1
3,Basic reproductive number of covid19,Machine learning identifies basic and effectiv...,1,1,1
4,Basic reproductive number of covid19,• R 0 and R e : The basic reproduction number ...,1,1,0
...,...,...,...,...,...
251,Ways of covid19 transmission,"In the USA, past influenza pandemics have spre...",1,0,0
252,Ways of covid19 transmission,Indirect transmission via non-contact route ca...,1,1,1
253,Ways of covid19 transmission,Only half of the students stayed regularly upd...,0,0,0
254,Ways of covid19 transmission,COVID-19 is featured by a long incubation peri...,1,1,1


In [152]:
c = pd.read_csv('/Users/patrik/PycharmProjects/TAR-project/data/remaining.csv')
c = c[['query', 'text', 'niksa', 'cvite', 'paco']]
c

,query,text,niksa,cvite,paco
0,Comorbidities and coinfections with effect on ...,The recent outbreak of Covid19 has required ur...,0,0,0
1,Comorbidities and coinfections with effect on ...,"At first glance, the predicted numbers of prev...",1,0,1
2,Comorbidities and coinfections with effect on ...,The fourth uncertainty is the duration of the ...,0,0,0
3,Comorbidities and coinfections with effect on ...,Pre-existing lung disease and diabetes were co...,1,1,1
4,Covid19 effect on pregnancy,4. There are repeated waves of COVID19 until a...,0,0,0
5,Covid19 effect on pregnancy,The recent outbreak of Covid19 has required ur...,0,0,0
6,Covid19 effect on pregnancy,The fourth uncertainty is the duration of the ...,0,0,0
7,Covid19 effect on pregnancy,A simplified model of Covid19 epidemic dynamic...,0,0,0
8,Covid19 effect on pregnancy,• #VIRUS: Hashtags which relate only to Corona...,0,0,0
9,Covid19 effect on pregnancy,Pre-existing lung disease and diabetes were co...,0,0,0


In [153]:
d = pd.concat([b, c])
d

,query,text,niksa,cvite,paco
0,Basic reproductive number of covid19,. The next-generation matrix will be used to d...,1,1,1
1,Basic reproductive number of covid19,⟨ Figure 1 is near here ⟩ According to the con...,1,1,0
2,Basic reproductive number of covid19,Studies such as [3] show that the effectivenes...,1,1,1
3,Basic reproductive number of covid19,Machine learning identifies basic and effectiv...,1,1,1
4,Basic reproductive number of covid19,• R 0 and R e : The basic reproduction number ...,1,1,0
...,...,...,...,...,...
50,Ways of covid19 transmission,The recent outbreak of Covid19 has required ur...,0,0,0
51,Ways of covid19 transmission,Thermal screening at airport has 54% COVID19 d...,0,0,0
52,Ways of covid19 transmission,On 24 th March 2020 the UK introduced a range ...,0,0,0
53,Ways of covid19 transmission,"As the COVID19 outbreak unfolds, prevention an...",1,1,1


In [154]:
e = pd.merge(a, d, on=['query', 'text']) # TODO: negdje se igubi njih 5??
e

,id,query,text,sim,model,niksa,cvite,paco
0,ccfe1455524bb1da264c8d868ecb5247ba869508,Basic reproductive number of covid19,. The next-generation matrix will be used to d...,0.815270,d2v,1,1,1
1,8c6e523654a7eab291bd150a6894986f76a30b4f,Basic reproductive number of covid19,⟨ Figure 1 is near here ⟩ According to the con...,0.920774,w2v_tf-idm,1,1,0
2,8c6e523654a7eab291bd150a6894986f76a30b4f,Basic reproductive number of covid19,⟨ Figure 1 is near here ⟩ According to the con...,0.803996,w2v,1,1,0
3,6b4ec0cb89bcb5a29dfb2328792ce64ba06ac553,Basic reproductive number of covid19,Studies such as [3] show that the effectivenes...,0.915905,w2v_tf-idm,1,1,1
4,6b4ec0cb89bcb5a29dfb2328792ce64ba06ac553,Basic reproductive number of covid19,Studies such as [3] show that the effectivenes...,0.436640,tf-idf,1,1,1
...,...,...,...,...,...,...,...,...
395,859eb40d59630f3d3ec9f0ded4f803098eee0fb3,Ways of covid19 transmission,Thermal screening at airport has 54% COVID19 d...,0.266427,bm25,0,0,0
396,9f28ffc6d0ffd16df162075e55d9a76e09226957,Ways of covid19 transmission,On 24 th March 2020 the UK introduced a range ...,0.349203,tf-idf,0,0,0
397,9275fca658701f7e6af4f0514ca2a95f3fd2c5cd,Ways of covid19 transmission,"As the COVID19 outbreak unfolds, prevention an...",0.361590,tf-idf,1,1,1
398,670790eb6d158a57450705bcd87218bf479ed3be,Ways of covid19 transmission,The pandemic of COVID19 is ongoing. Till April...,0.401133,tf-idf,1,1,1


In [158]:
e.to_csv('/Users/patrik/PycharmProjects/TAR-project/data/annotated_without_bert.csv')

In [168]:
bert = pd.read_csv('/Users/patrik/PycharmProjects/TAR-project/data/roBERTa_results.csv')
bert = bert[bert.columns[1:]]
bert['model'] = 'roBERTa'
bert

,id,query,text,sim,niksa,cvite,paco,model
0,7fb7c0c48b30e66dfaea5fdec16d999ed52d1ee0,Basic reproductive number of covid19,"Following [2, 3] , we assumed that number of s...",0.816147,1,1,0,roBERTa
1,cfb5e69aa2aa5a2a0a42768ce86a2275824a5417,Basic reproductive number of covid19,Countries confronting the COVID-19 pandemic ar...,0.808927,0,1,0,roBERTa
2,bc49552ce19a7c47d89afe7b98cc330c54da9d49,Basic reproductive number of covid19,COVID-19 is caused by the virus SARS-CoV-2. Cl...,0.800297,1,1,1,roBERTa
3,bbf09194127619f57b3ddf5daf684593a5831367,Basic reproductive number of covid19,"COVID-19, initially observed/detected in Hubei...",0.788188,1,1,1,roBERTa
4,245319b9ff57c43cd0d2d027218c842a7c87ce3a,Basic reproductive number of covid19,Transmission occurred only within the environm...,0.785228,1,1,1,roBERTa
...,...,...,...,...,...,...,...,...
75,16f1fab1289587610e879c3d39ec74c12d42c651,Ways of covid19 transmission,The transmission route for COVID-19 is shown i...,0.864776,1,0,1,roBERTa
76,e87480778faffce009192a7d51ca0b9b66038712,Ways of covid19 transmission,Background: Despite efforts aimed at forecasti...,0.864011,1,1,1,roBERTa
77,d3bf2b6f10dbe7ba6f218a8d5696b9e6bb7adb44,Ways of covid19 transmission,Objective: To estimate the number of deaths in...,0.860813,0,0,0,roBERTa
78,5139670b3f350f49dafcb766516ead7de5b23ca1,Ways of covid19 transmission,We conducted nucleic acid testing for a total ...,0.860759,0,0,0,roBERTa


In [169]:
all_results = pd.concat([e, bert])
all_results

,id,query,text,sim,model,niksa,cvite,paco
0,ccfe1455524bb1da264c8d868ecb5247ba869508,Basic reproductive number of covid19,. The next-generation matrix will be used to d...,0.815270,d2v,1,1,1
1,8c6e523654a7eab291bd150a6894986f76a30b4f,Basic reproductive number of covid19,⟨ Figure 1 is near here ⟩ According to the con...,0.920774,w2v_tf-idm,1,1,0
2,8c6e523654a7eab291bd150a6894986f76a30b4f,Basic reproductive number of covid19,⟨ Figure 1 is near here ⟩ According to the con...,0.803996,w2v,1,1,0
3,6b4ec0cb89bcb5a29dfb2328792ce64ba06ac553,Basic reproductive number of covid19,Studies such as [3] show that the effectivenes...,0.915905,w2v_tf-idm,1,1,1
4,6b4ec0cb89bcb5a29dfb2328792ce64ba06ac553,Basic reproductive number of covid19,Studies such as [3] show that the effectivenes...,0.436640,tf-idf,1,1,1
...,...,...,...,...,...,...,...,...
75,16f1fab1289587610e879c3d39ec74c12d42c651,Ways of covid19 transmission,The transmission route for COVID-19 is shown i...,0.864776,roBERTa,1,0,1
76,e87480778faffce009192a7d51ca0b9b66038712,Ways of covid19 transmission,Background: Despite efforts aimed at forecasti...,0.864011,roBERTa,1,1,1
77,d3bf2b6f10dbe7ba6f218a8d5696b9e6bb7adb44,Ways of covid19 transmission,Objective: To estimate the number of deaths in...,0.860813,roBERTa,0,0,0
78,5139670b3f350f49dafcb766516ead7de5b23ca1,Ways of covid19 transmission,We conducted nucleic acid testing for a total ...,0.860759,roBERTa,0,0,0


In [172]:
all_results.model = all_results.model.replace(to_replace={'w2v_tf-idm': 'w2v_tf-idf'})
all_results

,id,query,text,sim,model,niksa,cvite,paco
0,ccfe1455524bb1da264c8d868ecb5247ba869508,Basic reproductive number of covid19,. The next-generation matrix will be used to d...,0.815270,d2v,1,1,1
1,8c6e523654a7eab291bd150a6894986f76a30b4f,Basic reproductive number of covid19,⟨ Figure 1 is near here ⟩ According to the con...,0.920774,w2v_tf-idf,1,1,0
2,8c6e523654a7eab291bd150a6894986f76a30b4f,Basic reproductive number of covid19,⟨ Figure 1 is near here ⟩ According to the con...,0.803996,w2v,1,1,0
3,6b4ec0cb89bcb5a29dfb2328792ce64ba06ac553,Basic reproductive number of covid19,Studies such as [3] show that the effectivenes...,0.915905,w2v_tf-idf,1,1,1
4,6b4ec0cb89bcb5a29dfb2328792ce64ba06ac553,Basic reproductive number of covid19,Studies such as [3] show that the effectivenes...,0.436640,tf-idf,1,1,1
...,...,...,...,...,...,...,...,...
75,16f1fab1289587610e879c3d39ec74c12d42c651,Ways of covid19 transmission,The transmission route for COVID-19 is shown i...,0.864776,roBERTa,1,0,1
76,e87480778faffce009192a7d51ca0b9b66038712,Ways of covid19 transmission,Background: Despite efforts aimed at forecasti...,0.864011,roBERTa,1,1,1
77,d3bf2b6f10dbe7ba6f218a8d5696b9e6bb7adb44,Ways of covid19 transmission,Objective: To estimate the number of deaths in...,0.860813,roBERTa,0,0,0
78,5139670b3f350f49dafcb766516ead7de5b23ca1,Ways of covid19 transmission,We conducted nucleic acid testing for a total ...,0.860759,roBERTa,0,0,0


In [173]:
all_results.to_csv('/Users/patrik/PycharmProjects/TAR-project/data/annotated_all.csv')

In [178]:
blje = all_results[['niksa', 'cvite', 'paco']].sum(axis=1)

In [181]:
blje[blje<2] = 0

In [183]:
blje[blje>=2] = 1

In [185]:
blje

0     1
1     1
2     1
3     1
4     1
     ..
75    1
76    1
77    0
78    0
79    0
Length: 480, dtype: int64

In [186]:
all_results['relevance'] = blje

In [188]:
all_results.to_csv('/Users/patrik/PycharmProjects/TAR-project/data/annotated_all.csv')

In [13]:
all_results = pd.read_csv('/Users/patrik/PycharmProjects/TAR-project/data/annotated_all.csv')
all_results = all_results[all_results.columns[1:]]
all_results

,id,query,text,sim,model,niksa,cvite,paco,relevance
0,ccfe1455524bb1da264c8d868ecb5247ba869508,Basic reproductive number of covid19,. The next-generation matrix will be used to d...,0.815270,d2v,1,1,1,1
1,8c6e523654a7eab291bd150a6894986f76a30b4f,Basic reproductive number of covid19,⟨ Figure 1 is near here ⟩ According to the con...,0.920774,w2v_tf-idf,1,1,0,1
2,8c6e523654a7eab291bd150a6894986f76a30b4f,Basic reproductive number of covid19,⟨ Figure 1 is near here ⟩ According to the con...,0.803996,w2v,1,1,0,1
3,6b4ec0cb89bcb5a29dfb2328792ce64ba06ac553,Basic reproductive number of covid19,Studies such as [3] show that the effectivenes...,0.915905,w2v_tf-idf,1,1,1,1
4,6b4ec0cb89bcb5a29dfb2328792ce64ba06ac553,Basic reproductive number of covid19,Studies such as [3] show that the effectivenes...,0.436640,tf-idf,1,1,1,1
...,...,...,...,...,...,...,...,...,...
475,16f1fab1289587610e879c3d39ec74c12d42c651,Ways of covid19 transmission,The transmission route for COVID-19 is shown i...,0.864776,roBERTa,1,0,1,1
476,e87480778faffce009192a7d51ca0b9b66038712,Ways of covid19 transmission,Background: Despite efforts aimed at forecasti...,0.864011,roBERTa,1,1,1,1
477,d3bf2b6f10dbe7ba6f218a8d5696b9e6bb7adb44,Ways of covid19 transmission,Objective: To estimate the number of deaths in...,0.860813,roBERTa,0,0,0,0
478,5139670b3f350f49dafcb766516ead7de5b23ca1,Ways of covid19 transmission,We conducted nucleic acid testing for a total ...,0.860759,roBERTa,0,0,0,0


In [21]:
all_results['text_len']  = all_results['text'].str.len()

In [22]:
all_results

,id,query,text,sim,model,niksa,cvite,paco,relevance,text_len
0,ccfe1455524bb1da264c8d868ecb5247ba869508,Basic reproductive number of covid19,. The next-generation matrix will be used to d...,0.815270,d2v,1,1,1,1,269.0
1,8c6e523654a7eab291bd150a6894986f76a30b4f,Basic reproductive number of covid19,⟨ Figure 1 is near here ⟩ According to the con...,0.920774,w2v_tf-idf,1,1,0,1,337.0
2,8c6e523654a7eab291bd150a6894986f76a30b4f,Basic reproductive number of covid19,⟨ Figure 1 is near here ⟩ According to the con...,0.803996,w2v,1,1,0,1,337.0
3,6b4ec0cb89bcb5a29dfb2328792ce64ba06ac553,Basic reproductive number of covid19,Studies such as [3] show that the effectivenes...,0.915905,w2v_tf-idf,1,1,1,1,517.0
4,6b4ec0cb89bcb5a29dfb2328792ce64ba06ac553,Basic reproductive number of covid19,Studies such as [3] show that the effectivenes...,0.436640,tf-idf,1,1,1,1,517.0
...,...,...,...,...,...,...,...,...,...,...
475,16f1fab1289587610e879c3d39ec74c12d42c651,Ways of covid19 transmission,The transmission route for COVID-19 is shown i...,0.864776,roBERTa,1,0,1,1,144.0
476,e87480778faffce009192a7d51ca0b9b66038712,Ways of covid19 transmission,Background: Despite efforts aimed at forecasti...,0.864011,roBERTa,1,1,1,1,2300.0
477,d3bf2b6f10dbe7ba6f218a8d5696b9e6bb7adb44,Ways of covid19 transmission,Objective: To estimate the number of deaths in...,0.860813,roBERTa,0,0,0,0,875.0
478,5139670b3f350f49dafcb766516ead7de5b23ca1,Ways of covid19 transmission,We conducted nucleic acid testing for a total ...,0.860759,roBERTa,0,0,0,0,266.0


In [27]:
pd.DataFrame(all_results.groupby(['query', 'model'])['text_len'].sum())

text_len
query                                              model               
Basic reproductive number of covid19               bm25          4353.0
                                                   d2v           4853.0
                                                   roBERTa       9116.0
                                                   tf-idf        8689.0
                                                   w2v           4296.0
                                                   w2v_tf-idf   11325.0
Comorbidities and coinfections with effect on t... bm25          7412.0
                                                   d2v           5408.0
                                                   roBERTa       6209.0
                                                   tf-idf       10283.0
                                                   w2v          10697.0
                                                   w2v_tf-idf    8727.0
Covid19 effect on pregnancy                        bm25          2353.0
                                                   d2v           6679.0
                                                   roBERTa      11782.0
                                                   tf-idf        9520.0
                                                   w2v           9445.0
                                                   w2v_tf-idf    9300.0
Incubation period of covid19                       bm25          3720.0
                                                   d2v           5965.0
                                                   roBERTa       8079.0
                                                   tf-idf        9317.0
                                                   w2v           7334.0
                                                   w2v_tf-idf    8941.0
Mortality rate of covid19                          bm25          2698.0
                                                   d2v           5305.0
                                                   roBERTa       9377.0
                                                   tf-idf        8515.0
                                                   w2v           4498.0
                                                   w2v_tf-idf    5949.0
Smoking effect on the severity of covid19          bm25          4149.0
                                                   d2v           5657.0
                                                   roBERTa       8727.0
                                                   tf-idf       16033.0
                                                   w2v           7833.0
                                                   w2v_tf-idf    6809.0
Temperature effect on covid19                      bm25          4096.0
                                                   d2v           7261.0
                                                   roBERTa       7345.0
                                                   tf-idf        7038.0
                                                   w2v           7106.0
                                                   w2v_tf-idf    9890.0
Ways of covid19 transmission                       bm25          2426.0
                                                   d2v           5318.0
                                                   roBERTa       7605.0
                                                   tf-idf       10432.0
                                                   w2v           8243.0
                                                   w2v_tf-idf   12787.0

In [28]:
pd.DataFrame(all_results.groupby('model')['text_len'].sum())

,text_len
model,
bm25,31207.0
d2v,46446.0
roBERTa,68240.0
tf-idf,79827.0
w2v,59452.0
w2v_tf-idf,73728.0


In [44]:
all_results[['query', 'text']].drop_duplicates()

,query,text
0,Basic reproductive number of covid19,. The next-generation matrix will be used to d...
1,Basic reproductive number of covid19,⟨ Figure 1 is near here ⟩ According to the con...
3,Basic reproductive number of covid19,Studies such as [3] show that the effectivenes...
6,Basic reproductive number of covid19,Machine learning identifies basic and effectiv...
7,Basic reproductive number of covid19,• R 0 and R e : The basic reproduction number ...
...,...,...
475,Ways of covid19 transmission,The transmission route for COVID-19 is shown i...
476,Ways of covid19 transmission,Background: Despite efforts aimed at forecasti...
477,Ways of covid19 transmission,Objective: To estimate the number of deaths in...
478,Ways of covid19 transmission,We conducted nucleic acid testing for a total ...


In [8]:
all_results.groupby('model')['relevance'].sum()

model
bm25          38
d2v           46
roBERTa       45
tf-idf        41
w2v           72
w2v_tf-idf    66
Name: relevance, dtype: int64

In [9]:
all_results[['niksa', 'cvite', 'paco']].sum()

niksa    313
cvite    307
paco     258
dtype: int64

In [10]:
a = pd.DataFrame(all_results.groupby(['query', 'model'])['relevance'].sum()).reset_index()

In [11]:
a

,query,model,relevance
0,Basic reproductive number of covid19,bm25,8
1,Basic reproductive number of covid19,d2v,4
2,Basic reproductive number of covid19,roBERTa,8
3,Basic reproductive number of covid19,tf-idf,5
4,Basic reproductive number of covid19,w2v,9
5,Basic reproductive number of covid19,w2v_tf-idf,10
6,Comorbidities and coinfections with effect on ...,bm25,4
7,Comorbidities and coinfections with effect on ...,d2v,6
8,Comorbidities and coinfections with effect on ...,roBERTa,5
9,Comorbidities and coinfections with effect on ...,tf-idf,4


In [13]:
b = a.pivot_table(index='query', columns='model', values='relevance')
b

model,bm25,d2v,roBERTa,tf-idf,w2v,w2v_tf-idf
query,,,,,,
Basic reproductive number of covid19,8,4,8,5,9,10
Comorbidities and coinfections with effect on the severity of covid19,4,6,5,4,9,10
Covid19 effect on pregnancy,4,5,9,3,7,2
Incubation period of covid19,7,7,1,8,10,9
Mortality rate of covid19,5,8,4,3,9,9
Smoking effect on the severity of covid19,8,7,10,9,10,9
Temperature effect on covid19,1,6,4,4,9,10
Ways of covid19 transmission,1,3,4,5,9,7


In [28]:
pd.concat([b,pd.DataFrame(b.sum()).T])

model,bm25,d2v,roBERTa,tf-idf,w2v,w2v_tf-idf
Basic reproductive number of covid19,8,4,8,5,9,10
Comorbidities and coinfections with effect on the severity of covid19,4,6,5,4,9,10
Covid19 effect on pregnancy,4,5,9,3,7,2
Incubation period of covid19,7,7,1,8,10,9
Mortality rate of covid19,5,8,4,3,9,9
Smoking effect on the severity of covid19,8,7,10,9,10,9
Temperature effect on covid19,1,6,4,4,9,10
Ways of covid19 transmission,1,3,4,5,9,7
0,38,46,45,41,72,66


In [21]:
c = pd.DataFrame(b.sum()).T

In [31]:
c.index = ['all queries']
c.index.name = 'query'
c

model,bm25,d2v,roBERTa,tf-idf,w2v,w2v_tf-idf
query,,,,,,
all queries,38,46,45,41,72,66


In [35]:
b/10

model,bm25,d2v,roBERTa,tf-idf,w2v,w2v_tf-idf
query,,,,,,
Basic reproductive number of covid19,0.8,0.4,0.8,0.5,0.9,1.0
Comorbidities and coinfections with effect on the severity of covid19,0.4,0.6,0.5,0.4,0.9,1.0
Covid19 effect on pregnancy,0.4,0.5,0.9,0.3,0.7,0.2
Incubation period of covid19,0.7,0.7,0.1,0.8,1.0,0.9
Mortality rate of covid19,0.5,0.8,0.4,0.3,0.9,0.9
Smoking effect on the severity of covid19,0.8,0.7,1.0,0.9,1.0,0.9
Temperature effect on covid19,0.1,0.6,0.4,0.4,0.9,1.0
Ways of covid19 transmission,0.1,0.3,0.4,0.5,0.9,0.7


In [37]:
acc = pd.concat([b/10, c/80])
acc

model,bm25,d2v,roBERTa,tf-idf,w2v,w2v_tf-idf
query,,,,,,
Basic reproductive number of covid19,0.800,0.400,0.8000,0.5000,0.9,1.000
Comorbidities and coinfections with effect on the severity of covid19,0.400,0.600,0.5000,0.4000,0.9,1.000
Covid19 effect on pregnancy,0.400,0.500,0.9000,0.3000,0.7,0.200
Incubation period of covid19,0.700,0.700,0.1000,0.8000,1.0,0.900
Mortality rate of covid19,0.500,0.800,0.4000,0.3000,0.9,0.900
Smoking effect on the severity of covid19,0.800,0.700,1.0000,0.9000,1.0,0.900
Temperature effect on covid19,0.100,0.600,0.4000,0.4000,0.9,1.000
Ways of covid19 transmission,0.100,0.300,0.4000,0.5000,0.9,0.700
all queries,0.475,0.575,0.5625,0.5125,0.9,0.825


In [39]:
counts = pd.concat([b, c])
counts

model,bm25,d2v,roBERTa,tf-idf,w2v,w2v_tf-idf
query,,,,,,
Basic reproductive number of covid19,8,4,8,5,9,10
Comorbidities and coinfections with effect on the severity of covid19,4,6,5,4,9,10
Covid19 effect on pregnancy,4,5,9,3,7,2
Incubation period of covid19,7,7,1,8,10,9
Mortality rate of covid19,5,8,4,3,9,9
Smoking effect on the severity of covid19,8,7,10,9,10,9
Temperature effect on covid19,1,6,4,4,9,10
Ways of covid19 transmission,1,3,4,5,9,7
all queries,38,46,45,41,72,66


In [40]:
counts.to_csv('/Users/patrik/PycharmProjects/TAR-project/data/counts.csv')

In [38]:
blje.to_csv('/Users/patrik/PycharmProjects/TAR-project/data/acc.csv')
